In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.datasets import cifar10

## **Load dataset**

In [2]:
(X_train, y_train), (X_test, y_test) = cifar10.load_data()
X_train.shape, X_test.shape, y_train.shape, y_test.shape

170498071/170498071 [==============================] - 379s 2us/step


((50000, 32, 32, 3), (10000, 32, 32, 3), (50000, 1), (10000, 1))

## **Normalization**

In [3]:
X_train = X_train.astype("float32") / 255.0
X_test = X_test.astype("float32") / 225.0
X_train.dtype, X_test.dtype

(dtype('float32'), dtype('float32'))

In [4]:
model_cifar10 = keras.Sequential(
    [
        keras.Input(shape=(32,32,3)), # we use convolutional network so do not reshape
        layers.Conv2D(32, 3, padding='valid', activation='relu'),
        layers.MaxPooling2D(pool_size=(2,2)),
    ]
)

model_cifar10.summary()

AttributeError: module 'keras.api._v2.keras.layers' has no attribute 'Conv2d'